# Assignment 2

## Data cleaning

#### Load data and find freature present in both approved and rejected data. 

In [1]:
import pandas as pd

approved_csv = pd.read_csv("LoanStats3a.csv", skiprows=1)
rejected_csv = pd.read_csv("RejectStatsA.csv", skiprows=1)

### Mapping ###
# 'loan_amnt' - 'Amount Requested'
# 'issue_d' - 'Application Date'
# 'purpose', 'title' - 'Loan Title'
# '' - 'Risk_Score' (no match)
# 'dti' - 'Debt-To-Income Ratio'
# 'zip_code' - 'Zip Code'
# 'addr_state' - 'State'
# 'policy_code' - 'Policy Code'
# 'emp_length' - 'Employment Length'
approved_raw_data = approved_csv[['loan_amnt','issue_d', 'purpose', 'dti', 'zip_code',
                                  'addr_state', 'policy_code','emp_length']]
rejected_raw_data = rejected_csv[['Amount Requested','Application Date', 'Loan Title',
                                  'Debt-To-Income Ratio', 'Zip Code', 'State',
                                  'Policy Code', 'Employment Length']]
rejected_raw_data.columns = ['loan_amnt','issue_d', 'purpose', 'dti', 'zip_code',
                             'addr_state', 'policy_code', 'emp_length']

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Count the number of entries avaliable in each set, and display a small sample.

In [2]:
print ("The dataset contain {:d} approved data.".format(approved_raw_data.shape[0]))
print ("The dataset contain {:d} rejected data.".format(rejected_raw_data.shape[0]))

print ("\n")
print ("Sample from approved data:")
print (approved_raw_data.sample(5))
print ("\n")
print ("Sample from rejected data:")
print (rejected_raw_data.sample(5))

The dataset contain 42538 approved data.
The dataset contain 755491 rejected data.


Sample from approved data:
       loan_amnt   issue_d             purpose    dti zip_code addr_state  \
6017      2500.0  Oct-2011               house  15.56    019xx         MA   
1241      8875.0  Dec-2011  debt_consolidation  28.84    770xx         TX   
34345     6500.0  Nov-2009               house   8.64    926xx         CA   
28751    25000.0  Jun-2010  debt_consolidation  22.63    923xx         CA   
21116     6000.0  Jan-2011             medical  22.28    926xx         CA   

       policy_code emp_length  
6017           1.0        NaN  
1241           1.0  10+ years  
34345          1.0     1 year  
28751          1.0    4 years  
21116          1.0    4 years  


Sample from rejected data:
        loan_amnt     issue_d                   purpose     dti zip_code  \
645090    11500.0  2012-10-02               credit_card  35.81%    799xx   
17860      7500.0  2008-06-17  Need to Consolidate D

#### Count how may entries have missing value for both set.

In [3]:
nan_count = pd.DataFrame({"approved": approved_raw_data.isnull().sum(),
                          "approved %": approved_raw_data.isnull().sum() / len(approved_raw_data), 
                          "rejected": rejected_raw_data.isnull().sum(),
                          "rejected %": rejected_raw_data.isnull().sum() / len(rejected_raw_data)})
print (nan_count)

             approved  approved %  rejected  rejected %
loan_amnt           3    0.000071         0    0.000000
issue_d             3    0.000071         0    0.000000
purpose             3    0.000071        14    0.000019
dti                 3    0.000071         0    0.000000
zip_code            3    0.000071        22    0.000029
addr_state          3    0.000071        21    0.000028
policy_code         3    0.000071         0    0.000000
emp_length       1115    0.026212      8130    0.010761


#### In general, the dataset is relatively complet, with most of the missing value occur in the 'emp_length' feature. However, it is still less than 3% of missing value for the approved set and about 1% for the rejected set. This means it is relatively safe to simply drop the entries with NaN value with low possibility of introducing a significant bias.

In [4]:
approved_raw_data = approved_raw_data.dropna()
rejected_raw_data = rejected_raw_data.dropna()

## Data processing

#### Feature 'issue_d' and 'dti' are coded in different format in the approved and rejected set. We will need to standardize the format.

In [5]:
import copy

# copy raw data into intermediate processing set
approved_data_0 = copy.deepcopy(approved_raw_data)
rejected_data_0 = copy.deepcopy(rejected_raw_data)

In [6]:
from datetime import datetime
import numpy as np

def date_string_to_datetime(row):
    try:   # for approved dataset
        return datetime.strptime(row['issue_d'], "%b-%Y")
    except:   # for rejected dataset
        return datetime.strptime(row['issue_d'], "%Y-%m-%d")

def extract_year_lambda(row):
    return date_string_to_datetime(row).year

def extract_month_lambda(row):
    return date_string_to_datetime(row).month

def dti_percent_to_decimal(row):
    return np.float64(row['dti'].strip("%"))
    

approved_data_0['issue_y'] = approved_data_0.apply(extract_year_lambda, axis=1)
approved_data_0['issue_m'] = approved_data_0.apply(extract_month_lambda, axis=1)
approved_data_0 = approved_data_0.drop(columns=['issue_d'])

rejected_data_0['issue_y'] = rejected_data_0.apply(extract_year_lambda, axis=1)
rejected_data_0['issue_m'] = rejected_data_0.apply(extract_month_lambda, axis=1)
rejected_data_0['dti'] = rejected_data_0.apply(dti_percent_to_decimal, axis=1)
rejected_data_0 = rejected_data_0.drop(columns=['issue_d'])

In [10]:
print ("Sample from approved data:")
print (approved_data_0.sample(5))
print ("\n")
print ("Sample from rejected data:")
print (rejected_data_0.sample(5))

Sample from approved data:
       loan_amnt             purpose    dti zip_code addr_state  policy_code  \
24026    10000.0         credit_card  15.58    115xx         NY          1.0   
30114     2500.0               other   9.75    756xx         TX          1.0   
4891      6250.0  debt_consolidation  16.72    708xx         LA          1.0   
41971    24000.0      small_business  18.38    921xx         CA          1.0   
39027    22000.0  debt_consolidation   9.81    100xx         NY          1.0   

      emp_length  issue_y  issue_m  
24026    4 years     2010       11  
30114   < 1 year     2010        5  
4891     2 years     2011       11  
41971    4 years     2008        2  
39027   < 1 year     2008        3  


Sample from rejected data:
        loan_amnt             purpose     dti zip_code addr_state  \
573083     2000.0  debt_consolidation    2.86    936xx         CA   
116524     1000.0             wedding    3.18    902xx         CA   
491444    15000.0          GREAT L

#### Feature 'purpose' is a string, containing discription of the intened use of the loan. We need to find a way to represent that data in a machine-friendly way.

In [13]:
print (approved_data_0.groupby('purpose').count())

                    loan_amnt    dti  zip_code  addr_state  policy_code  \
purpose                                                                   
car                      1563   1563      1563        1563         1563   
credit_card              5344   5344      5344        5344         5344   
debt_consolidation      19363  19363     19363       19363        19363   
educational               413    413       413         413          413   
home_improvement         3099   3099      3099        3099         3099   
house                     412    412       412         412          412   
major_purchase           2238   2238      2238        2238         2238   
medical                   726    726       726         726          726   
moving                    603    603       603         603          603   
other                    4259   4259      4259        4259         4259   
renewable_energy           98     98        98          98           98   
small_business           